In [1]:
# Imports
import os
import sys
sys.path.append(os.path.abspath(".."))   # one level up
import numpy as np
print(os.getcwd())
import cv2
import open3d as o3d
from scipy.spatial.transform import Rotation
from utils.lidar import PointCloud
from utils.camera import ImageData

d:\ODD - Connor\python-FRED\visualisation
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# User parameters
location = 'Cambogan'
sequence = '20250811_113017'
condition = 'flooded'
camera_pos = 'front'
image_timestamp = '20641521'
root_directory = f"../../Datasets/FRED/{condition}/KITTI-style"
# 01000000

############ Define filenames and directories ####################################

image_dir = f"{root_directory}/{location}_{sequence}/{camera_pos}-imgs/"
lidar_dir = f"{root_directory}/{location}_{sequence}/ouster/"
utm_dir = f"{root_directory}/{location}_{sequence}/utm/"

img_calib_file = f"../camera_calib.txt"
lidar_calib_file = f"../calib.txt"

image_filename = f"{image_dir}/{image_timestamp}.png"

############ Find closest point cloud and UTM position data ########################

lidar_timestamps = np.array([int(filename.split('.bin')[0]) for filename in os.listdir(lidar_dir) if os.path.isfile(lidar_dir+filename)])

closest_lidar = np.argmin(abs(lidar_timestamps-int(image_timestamp)))
timestamp_diff = abs(int(image_timestamp)-lidar_timestamps[closest_lidar])

timestamp_tolerance = 200000 # in microseconds (0.2 seconds)

if timestamp_diff > timestamp_tolerance:
    raise Exception("No pointcloud in close enough proximity")
else:
    lidar_filename = f"{lidar_dir}/{lidar_timestamps[closest_lidar]}.bin"


utm_timestamps = np.array([int(filename.split('.txt')[0]) for filename in os.listdir(utm_dir) if os.path.isfile(utm_dir+filename)])

closest_utm = np.argmin(abs(utm_timestamps-int(image_timestamp)))
timestamp_diff = abs(int(image_timestamp)-utm_timestamps[closest_utm])

if timestamp_diff > timestamp_tolerance:
    raise Exception("No utm measurment in close enough proximity")
else:
    utm_filename = f"{utm_dir}/{utm_timestamps[closest_utm]}.txt"

In [3]:
print(image_filename)
print(lidar_filename)
print(utm_filename)

../../Datasets/FRED/flooded/KITTI-style/Cambogan_20250811_113017/front-imgs//20641521.png
../../Datasets/FRED/flooded/KITTI-style/Cambogan_20250811_113017/ouster//20692777.bin
../../Datasets/FRED/flooded/KITTI-style/Cambogan_20250811_113017/utm//20710844.txt


In [4]:
image = ImageData(image_filename, img_calib_file)
pointcloud = PointCloud(lidar_filename, lidar_calib_file)

# print(pointcloud.points.shape)

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(pointcloud.points[:,:3])
# o3d.visualization.draw_geometries([pcd])

# # Project to image coordinates
# rvec = np.zeros(3)  # No additional rotation
# tvec = np.zeros(3)  # No additional translation

# image_points, _ = cv2.projectPoints(pointcloud.points_ouster_to_cam(), rvec, tvec, image.camera_matrix, image.dist_coeffs)

# image_points = image_points.reshape(-1, 2)

# # Filter points within image bounds
# h, w = image.image.shape[0], image.image.shape[1]
# valid_img_mask = ((image_points[:, 0] >= 0) & (image_points[:, 0] < w) &
#                     (image_points[:, 1] >= 0) & (image_points[:, 1] < h))

# points2project = image_points[valid_img_mask]

TypeError: read_calib_file() missing 1 required positional argument: 'path'